# Exercice 3   

**Question 1**: Modelisation du problème de plus court chemin comme problème du flot de coût minimum

$G=(V,E)$, source: $s \in V$, puit: $t \in V$

$a(u,v)$: cout pour passer de u à v, $f(u,v)=1$ si $(u,v) \in $ {plus court chemin} 

Programme linéaire:
\begin{align*}
\min&\sum_{(u,v)\in E} a(u,v)f(u,v)&\\
s.c.& \sum_{(u,v)\in E}f(u,v) = \sum_{(v,w)\in E}f(v,w) \quad \forall v \in V \setminus \{s, t\}& \text{, contrainte de conservation du flux}\\
    & \sum_{s,v \in E} f(s,v) >= 1 &\text{ , contrainte de flux sortant de la source}\\
    &f(u,v)\in (0,1) \quad \forall (u,v) \in E&
\end{align*}

**Question 2:** Application aux exemples

In [13]:
from gurobipy import *

In [14]:
couts_i1 = [{
        ('a','b'): 4,
        ('a','c'): 5,
        ('b','c'): 2,
        ('b','d'): 1,
        ('b','e'): 2,
        ('b','f'): 7,
        ('c','d'): 5,
        ('c','e'): 2,
        ('d','f'): 3,
        ('e','f'): 5
    },
    {
        ('a','b'): 3,
        ('a','c'): 1,
        ('b','c'): 1,
        ('b','d'): 4,
        ('b','e'): 2,
        ('b','f'): 5,
        ('c','d'): 1,
        ('c','e'): 7,
        ('d','f'): 2,
        ('e','f'): 2
    }
]

couts_i2 = [
    {
        ('a','b'): 5,
        ('a','c'): 10,
        ('a','d'): 2,
        ('b','c'): 4,
        ('b','d'): 1,
        ('b','e'): 4,
        ('c','e'): 3,
        ('c','f'): 1,
        ('d','c'): 1,
        ('d','f'): 3,
        ('e','g'): 1,
        ('f','g'): 1
    },
    {
        ('a','b'): 3,
        ('a','c'): 4,
        ('a','d'): 6,
        ('b','c'): 2,
        ('b','d'): 3,
        ('b','e'): 6,
        ('c','e'): 1,
        ('c','f'): 2,
        ('d','c'): 4,
        ('d','f'): 5,
        ('e','g'): 1,
        ('f','g'): 1
    }
]



In [15]:
def minFlow(couts):
    # trouver noeuds
    noeuds = set()
    for arc in couts:
        noeuds.update(arc)
    noeuds = list(noeuds)

    source = min(noeuds)
    puits = max(noeuds)
    source, puits

    model = Model()
    model.params.LogToConsole = False

    nombre_arcs = len(couts)
    f = model.addVars(couts.keys(), vtype = GRB.BINARY, name='f')
    model.setObjective(quicksum([f[i,j] * cout for (i,j),cout in couts.items()]), GRB.MINIMIZE)

    # contrainte de conservation du flux
    for v in noeuds:
        if v == source or v == puits:
            continue
        flux_entrant = quicksum([f[i,v] for i,j in couts if j == v])
        flux_sortant = quicksum([f[v,j] for i,j in couts if i == v])
        model.addConstr(flux_entrant == flux_sortant)

    #for arc, cout in couts.items():
        #model.addConstr(f[arc] <= cout )    # contrainte de capacité
        #model.addConstr(f[arc] >= 0)        # contrainte de non-negativité


    model.addConstr(quicksum([f[source, j] for i,j in couts if i == source]) >= 1)


    model.optimize()
    objets_selectiones = [(i,j) for i,j in couts if f[i,j].X > 0.5]
    print('Chemin: ', objets_selectiones,', coût: ', int(model.ObjVal))

In [16]:
for i, couts_instances in enumerate([couts_i1, couts_i2]):
    print(f"Plus court chemin de l'instance {i} : ")
    for j,couts in enumerate(couts_instances):
        print(f"Scénario {j}:")
        minFlow(couts)


Plus court chemin de l'instance 0 : 
Scénario 0:
Chemin:  [('a', 'b'), ('b', 'd'), ('d', 'f')] , coût:  8
Scénario 1:
Chemin:  [('a', 'c'), ('c', 'd'), ('d', 'f')] , coût:  4
Plus court chemin de l'instance 1 : 
Scénario 0:
Chemin:  [('a', 'd'), ('c', 'f'), ('d', 'c'), ('f', 'g')] , coût:  5
Scénario 1:
Chemin:  [('a', 'c'), ('c', 'e'), ('e', 'g')] , coût:  6


**Question 3:**

## maxmin
Programme linéaire pour $n$ scénarios:
\begin{align*}
\min\quad   & t&&\\
s.c.\quad   &t>=\sum_{(u,v)\in E} a_i(u,v)f(u,v)&i\in\{1,\ldots,n\}\\
            & \sum_{(u,v)\in E}f(u,v) = \sum_{(v,w)\in E}f(v,w) \quad \forall v \in V \setminus \{s, t\}& \text{, contrainte de conservation du flux}\\
            & \sum_{s,v \in E} f(s,v) >= 1 &\text{ , contrainte de flux sortant de la source}&\\
            &f(u,v)\in \{0,1\} \quad \forall (u,v) \in E&
\end{align*}

In [43]:
def minFlowMaxMin(valeurs_scenarios):
    # trouver noeuds
    # on suppose que tous les scénarios ont les mêmes arcs


    noeuds = set()
    for arc in valeurs_scenarios[0]:
        noeuds.update(arc)
    noeuds = list(noeuds)

    source = min(noeuds)
    puits = max(noeuds)
    source, puits

    model = Model()
    model.params.LogToConsole = False

    f = model.addVars(valeurs_scenarios[0].keys(), vtype = GRB.BINARY, name='f')
    t = model.addVar(vtype = GRB.INTEGER)


    model.setObjective(t, GRB.MINIMIZE)
   
    # contrainte de t
    for couts in valeurs_scenarios:
        model.addConstr(t >= quicksum([f[i,j] * cout for (i,j),cout in couts.items()]))

    # contrainte de conservation du flux
    for v in noeuds:
        if v == source or v == puits:
            continue
        flux_entrant = quicksum([f[i,v] for i,j in couts if j == v])
        flux_sortant = quicksum([f[v,j] for i,j in couts if i == v])
        model.addConstr(flux_entrant == flux_sortant)

    #for arc, cout in couts.items():
        #model.addConstr(f[arc] <= cout )    # contrainte de capacité
        #model.addConstr(f[arc] >= 0)        # contrainte de non-negativité


    model.addConstr(quicksum([f[source, j] for i,j in couts if i == source]) >= 1)


    model.optimize()

    objets_selectiones = [(i,j) for i,j in couts if f[i,j].X > 0.5]

    vecteur_z = []
    for v_s in valeurs_scenarios:
        vecteur_z.append(sum([v_s[i,j] for (i,j) in objets_selectiones]))

    print('Chemin: ', objets_selectiones,', coût: ', int(model.ObjVal), ' vecteur_z: ', vecteur_z)

In [44]:
for i, couts_instances in enumerate([couts_i1, couts_i2]):
    print(f"Plus court chemin de l'instance {i} : ")
    minFlowMaxMin(couts_instances)


Plus court chemin de l'instance 0 : 
Chemin:  [('a', 'b'), ('b', 'd'), ('d', 'f')] , coût:  9  vecteur_z:  [8, 9]
Plus court chemin de l'instance 1 : 
Chemin:  [('a', 'b'), ('b', 'e'), ('e', 'g')] , coût:  10  vecteur_z:  [10, 10]


## minmax regret
Programme linéaire pour $n$ scénarios:
\begin{align*}
\min\quad   & t&&\\
s.c.\quad   &t>=\sum_{(u,v)\in E} a_i(u,v)f(u,v)&i\in\{1,\ldots,n\}\\
            & \sum_{(u,v)\in E}f(u,v) = \sum_{(v,w)\in E}f(v,w) \quad \forall v \in V \setminus \{s, t\}& \text{, contrainte de conservation du flux}\\
            & \sum_{s,v \in E} f(s,v) >= 1 &\text{ , contrainte de flux sortant de la source}&\\
            &f(u,v)\in \{0,1\} \quad \forall (u,v) \in E&
\end{align*}